In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time
from tqdm import tqdm as ProgressBar
import six # needed for Google Cloud client
import sys
import pickle
import operator
import nltk
import zlib
import cPickle
import parsing_util
import cleaning_util

import en # NodeBox https://www.nodebox.net/code/index.php/Linguistics#verb_conjugation

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [2]:

df=pd.read_pickle('CleanedIngredients.pkl')
df2=df.set_index("url")
ingredientsDict=pd.read_pickle('unique_ingredients.pkl')

In [ ]:
df["ingredients"].sample(5)

In [ ]:
df.sample(1)

In [5]:
# Iterate through df parse ingredients into a compressed dictionary, and save on disk.
client = language.LanguageServiceClient()
IngredientsDict=defaultdict()
IngredientsDict=pd.read_pickle('parsedIngredients.pkl')  
added=0
#df2=df.sample(1)
for idx, row in ProgressBar(df.iterrows(), desc="Processing ingredients"):
    if row["url"] in IngredientsDict: continue
    # print 'loading ' + row["url"]   
    ingredients = '.'.join(ingredient for ingredient in row['ingredients'])
    if isinstance(ingredients, six.binary_type):
        ingredients = ingredients.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=ingredients,
        type=enums.Document.Type.PLAIN_TEXT)

        # Detects syntax in the document. You can also analyze HTML with:
        #   document.type == enums.Document.Type.HTML
    #tokens = client.analyze_syntax(document).tokens
    tokens = client.annotate_text(document, {'extract_syntax': True,}).tokens
    tokensList=list(tokens)
    IngredientsDict[row['url']]=zlib.compress(cPickle.dumps(tokensList))
    added += 1
    if added > 10000:
        with open('parsedIngredients.pkl', 'wb') as f:
            pickle.dump(IngredientsDict, f) 
        added = 0
    
with open('parsedIngredients.pkl', 'wb') as f:
    pickle.dump(IngredientsDict, f)
    



Processing ingredients: 104680it [00:08, 12637.00it/s]


In [4]:
#IngredientsDict=pd.read_pickle('parsedIngredients.pkl')  
print len(IngredientsDict)
#for key,value in ingredientsDict.iteritems():
#    print key
#with open('parsedIngredients.pkl', 'wb') as f:
#    pickle.dump(IngredientsDict, f)

48417


In [7]:
# Iterate through df parse instructions into a compressed dictionary, and save on disk.
client = language.LanguageServiceClient()
InstructionsDict=defaultdict()
InstructionsDict=pd.read_pickle('parsedInstructions.pkl')  
added=0

for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
    if row["url"] in InstructionsDict: continue
    # print 'loading ' + row["url"]   
    instructions = '.'.join(instruction for instruction in row['instructionSteps'])
    if isinstance(instructions, six.binary_type):
        instructions = instructions.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=instructions,
        type=enums.Document.Type.PLAIN_TEXT)

        # Detects syntax in the document. You can also analyze HTML with:
        #   document.type == enums.Document.Type.HTML
    #tokens = client.analyze_syntax(document).tokens
    tokens = client.annotate_text(document, {'extract_syntax': True,}).tokens
    tokensList=list(tokens)
    InstructionsDict[row['url']]=zlib.compress(cPickle.dumps(tokensList))
    added += 1
    if added > 10000:
        with open('parsedInstructions.pkl', 'wb') as f:
            pickle.dump(InstructionsDict, f) 
        added = 0
    
with open('parsedInstructions.pkl', 'wb') as f:
    pickle.dump(InstructionsDict, f)
    
#ingredientsDict=pd.read_pickle('parsedInstruction.pkl')    
#for key,value in ingredientsDict.iteritems():
#    print key


Processing recipes: 104680it [03:05, 563.55it/s] 


In [9]:
CleanIngredientsDict=pd.read_pickle('parsedInstructions.pkl')  
print len(IngredientsDict)
#for key,value in ingredientsDict.iteritems():
#    print key
#with open('parsedIngredients.pkl', 'wb') as f:
#    pickle.dump(IngredientsDict, f)

48417


In [20]:
df2.loc['http://allrecipes.com/recipe/18080/almond-macaroon-nests/']["instructionSteps"]
#df.drop_duplicates(subset='url', keep='first', inplace=True)

[u'Preheat oven to 325 degrees F (165 degrees C). Grease cookie sheets.',
 u'Sift together the flour, baking powder and salt; set aside. In a medium bowl, cream together the butter, cream cheese, and sugar until smooth. Stir in the vanilla, then gradually beat in the dry ingredients.',
 u"Divide the coconut into three separate containers (see Cook's Note). Color one portion red, one yellow, and leave one plain. Toss the colored coconut together with the plain.",
 u'Roll cookie dough into walnut-sized balls, then roll the balls in the coconut mixture. Place them 2 inches apart onto the prepared cookie sheet. Press 1 candy into the center of each cookie.',
 u'Bake for 12 minutes in the preheated oven, then remove from the oven and press 2 more candies into the center of each cookie. Return to the oven to continue baking for another 4 to 5 minutes, until golden brown. Cool cookies on the baking sheet for a few minutes before removing to a wire rack to cool completely.']

In [16]:
df.shape

(48417, 11)

In [22]:
insDict=parsing_util.Instructions()
url='http://allrecipes.com/recipe/18080/almond-macaroon-nests/'
instructions=df2.loc[url]["instructionSteps"]
parsedInstructions = insDict.parse(url, instructions)
print parsedInstructions
insDict.close()

[text {
  content: "Preheat"
  begin_offset: -1
}
part_of_speech {
  tag: DET
}
dependency_edge {
  head_token_index: 1
  label: DET
}
lemma: "Preheat"
, text {
  content: "oven"
  begin_offset: -1
}
part_of_speech {
  tag: NOUN
  number: SINGULAR
}
dependency_edge {
  head_token_index: 1
  label: ROOT
}
lemma: "oven"
, text {
  content: "to"
  begin_offset: -1
}
part_of_speech {
  tag: ADP
}
dependency_edge {
  head_token_index: 1
  label: PREP
}
lemma: "to"
, text {
  content: "325"
  begin_offset: -1
}
part_of_speech {
  tag: NUM
}
dependency_edge {
  head_token_index: 5
  label: NUM
}
lemma: "325"
, text {
  content: "degrees"
  begin_offset: -1
}
part_of_speech {
  tag: NOUN
  number: PLURAL
}
dependency_edge {
  head_token_index: 5
  label: NN
}
lemma: "degree"
, text {
  content: "F"
  begin_offset: -1
}
part_of_speech {
  tag: NOUN
  number: SINGULAR
}
dependency_edge {
  head_token_index: 2
  label: POBJ
}
lemma: "F"
, text {
  content: "("
  begin_offset: -1
}
part_of_speech 

In [23]:
# Iterate through df parse instructions into a compressed dictionary, and save on disk.
cleanedIngredientsDict=defaultdict()
cleanedIngredientsDict=pd.read_pickle('cleanerIngredients.pkl')  
insDict=parsing_util.Instructions()
added=0

for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
    url=row["url"]
    if url in cleanedIngredientsDict: continue
    # print 'loading ' + row["url"]   
    parsedInstructions = insDict.parse(url, row["instructionSteps"])
    cleanIng = parsing_util.get_ingredients(parsedInstructions, row["ingredients"], ingredientsDict,0)
    cleanedIngredientsDict[url] = cleanIng
    added += 1
    if added > 10000:
        with open('cleanerIngredients.pkl', 'wb') as f:
            pickle.dump(cleanedIngredientsDict, f) 
        added = 0
    
with open('cleanerIngredients.pkl', 'wb') as f:
    pickle.dump(cleanedIngredientsDict, f)
insDict.close()    
#ingredientsDict=pd.read_pickle('parsedInstruction.pkl')    
#for key,value in ingredientsDict.iteritems():
#    print key


Processing recipes: 48417it [19:13, 41.96it/s]   


In [ ]:
CleanIngredientsDict=pd.read_pickle('cleanerIngredients.pkl')  
print len(CleanIngredientsDict)
#for key,value in ingredientsDict.iteritems():
#    print key
#with open('parsedIngredients.pkl', 'wb') as f:
#    pickle.dump(IngredientsDict, f)
for key,item in sorted(CleanIngredientsDict.items(), reverse=False, key=operator.itemgetter(1))[:5]:
    print item

In [ ]:
# Count ingredients. prepare a clean list 
UniqueIngredientsDict=defaultdict(int)
cleanedIngredientsDict=pd.read_pickle('cleanerIngredients.pkl')  
#ingredientsDict=pd.read_pickle('unique_ingredients.pkl')

for url,ingredients in cleanedIngredientsDict.items():
    for ingredient in ingredients:
        UniqueIngredientsDict[ingredient] += 1
    #print ingredient

for url,ingredients in sorted(cleanedIngredientsDict.items(), reverse=False, key=operator.itemgetter(1))[:10]:
    print url,ingredients
    

## Debugging code blow

In [ ]:
# For debugging
reload(parsing_util)
url='http://allrecipes.com/recipe/18080/almond-macaroon-nests/'

ins=df2.loc[url,'instructionSteps']
Instructions=ins.sample(1).values[0]
ing=df2.loc[url,'ingredients']
ingredients=ing.sample(1).values[0]
#Instructions="[u'Whisk the paprika, garlic, Italian seasoning, lemon juice, olive oil, pepper, basil, and brown sugar together in a bowl until thoroughly blended. Stir in the shrimp, and toss to evenly coat with the marinade. Cover and refrigerate at least 2 hours, turning once.', u'Preheat an outdoor grill for medium-high heat. Lightly oil grill grate, and place about 4 inches from heat source.', u'Remove shrimp from marinade, drain excess, and discard marinade.', u'Place shrimp on preheated grill and cook, turning once, until opaque in the center, 5 to 6 minutes. Serve immediately.']"
#ingredients=[u'2 teaspoons ground paprika', u'2 tablespoons fresh minced garlic', u'2 teaspoons Italian seasoning blend', u'2 tablespoons fresh lemon juice', u'1/4 cup olive oil', u'1/2 teaspoon ground black pepper', u'2 teaspoons dried basil leaves', u'2 tablespoons brown sugar, packed', u'2 pounds large shrimp (21-25 per pound), peeled and deveined', u'Add all ingredients to list', u'Add all ingredients to list']
parsedInstructions = insDict.parse(url, Instructions)

print Instructions
print ingredients
print parsing_util.get_ingredients(parsedInstructions, ingredients, ingredientsDict, 10)

In [ ]:
for idx, row in df.sample(3).iterrows():
    parsedInstructions = insDict.parse(row["url"], row["instructionSteps"])
    cleanIng = get_ingredients(parsedInstructions, row["ingredients"], ingredientsDict,2)
    print "url='"+row["url"]+"'"
    print 'Instructions="'+str(row["instructionSteps"])+'"'
    print 'ingredients='+str(row["ingredients"])
    # print cleanIng
    print
insDict.close()

In [12]:
# For debugging: find fro which recipe "yellow" came from 
for url,ingredients in cleanedIngredientsDict.items():
    if 'yellow' in ingredients:
        print url,ingredients

http://allrecipes.com/recipe/18080/almond-macaroon-nests/ set([u'butter', u'coconut', 'red food coloring', u'flour', u'vanilla', u'salt', u'yellow', u'sugar', u'baking powder', 'chocolate coated peanut', u'cream'])
